<div style="color: #2590c2; text-align: center;">
<span style="font-size:18pt;"><b>ST: BIG DATA ANALYTICS</b></span><br/>
<span>CS 696-16 (Fall'18)</span><br/><br/>
<span><b>Project 1</b></span><br/><br/>
<span>Submitted By</span><br/>
<span>Ashok Kumar Shrestha</span>
</div>

In [1]:
# Datasets from Kaggle
# Challenges in Representation Learning: Facial Expression Recognition Challenge
# source: https://www.kaggle.com/c/challenges-in-representation-learning-facial-expression-recognition-challenge/data

In [2]:
import csv, sys
import matplotlib.pyplot as plt
import numpy as np

def read_data():
    filename = 'fer2013.csv'
    cnt = 0
    data = []
    label = []
    w, h = 48, 48
    image = np.zeros((h, w), dtype=np.uint8)
    
    with open(filename, 'r') as f:
        reader = csv.reader(f)
        try:
            for row in reader:
                cnt += 1
                if cnt>1:
                    emotion = int(row[0]) - 1
                    pixels = list(map(int, row[1].split()))
                    pixels_array = np.asarray(pixels)
                    
                    data.append(pixels_array)
                    label.append(emotion)
                    #if cnt>10:
                    #    return data1, label
        
            return data, label
    
        except csv.Error as e:
            sys.exit('file %s, line %d: %s' % (filename, reader.line_num, e))


data, label = read_data()

In [3]:
X = np.array(data)[:100]
y = np.asarray(label)[:100]

#X = np.reshape(X,(len(X),48,48,1))
#y = np.reshape(label,(len(label),1))

print(X.shape, y.shape)

(100, 2304) (100,)


In [4]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)


(67, 2304) (33, 2304) (67,) (33,)


In [5]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 128
num_classes = 6
epochs = 1

# input image dimensions
img_rows, img_cols = 48, 48

# the data, split between train and test sets
#(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


x_train shape: (67, 48, 48, 1)
67 train samples
33 test samples
Train on 67 samples, validate on 33 samples
Epoch 1/1
67/67 [==============================] - 1s 17ms/step - loss: 1.8126 - acc: 0.1493 - val_loss: 1.9883 - val_acc: 0.0606
Test loss: 1.9883411580866033
Test accuracy: 0.06060606060606061
